# BNL Development Notebook

This notebook is for developing and testing the BNL package.


In [59]:
%load_ext autoreload
%autoreload 2

import bnl

# R2_BUCKET_PUBLIC_URL = "https://pub-05e404c031184ec4bbf69b0c2321b98e.r2.dev"

# slm_ds = bnl.data.Dataset(manifest_path=f"{R2_BUCKET_PUBLIC_URL}/manifest_cloud_boolean.csv")
slm_ds = bnl.data.Dataset(manifest_path="~/data/salami/metadata.csv")
# slm_ds = bnl.data.Dataset()
track = slm_ds[11]
est = track.load_annotation("adobe-mu1gamma1")
ref = track.load_annotation("reference")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
ref.plot().show()
est.plot().show()


In [84]:
# First figure: we need to show for T-measure, what's it's deal with monotonicity.
import mir_eval
import numpy as np

aligned_est = ref.align_layers(est)
mono_est_count = aligned_est.to_contour('count').to_hierarchy().to_multisegment()
mono_est_depth = aligned_est.to_contour('depth').to_hierarchy().to_multisegment()
mono_est_prob = aligned_est.to_contour('prob').to_hierarchy().to_multisegment()

a = mir_eval.hierarchy.tmeasure(ref.itvls, aligned_est.itvls)
b = mir_eval.hierarchy.tmeasure(ref.itvls, mono_est_depth.itvls)
c = mir_eval.hierarchy.tmeasure(ref.itvls, mono_est_count.itvls)
d = mir_eval.hierarchy.tmeasure(ref.itvls, mono_est_prob.itvls)
print(np.array([a, b, c, d]))

/Users/tomxi/code/bnl/.venv/lib/python3.12/site-packages/mir_eval/hierarchy.py:460: UserWarning:

Segment hierarchy is inconsistent at level 6

/Users/tomxi/code/bnl/.venv/lib/python3.12/site-packages/mir_eval/hierarchy.py:460: UserWarning:

Segment hierarchy is inconsistent at level 7

/Users/tomxi/code/bnl/.venv/lib/python3.12/site-packages/mir_eval/hierarchy.py:460: UserWarning:

Segment hierarchy is inconsistent at level 8

/Users/tomxi/code/bnl/.venv/lib/python3.12/site-packages/mir_eval/hierarchy.py:460: UserWarning:

Segment hierarchy is inconsistent at level 9

/Users/tomxi/code/bnl/.venv/lib/python3.12/site-packages/mir_eval/hierarchy.py:460: UserWarning:

Segment hierarchy is inconsistent at level 10

/Users/tomxi/code/bnl/.venv/lib/python3.12/site-packages/mir_eval/hierarchy.py:460: UserWarning:

Segment hierarchy is inconsistent at level 11



[[0.61329638 0.34236616 0.43942693]
 [0.28435288 0.52712114 0.36942258]
 [0.5417733  0.50039929 0.52026503]
 [0.45868704 0.52010819 0.48747047]]


In [ ]:
# Study the effect of monotonic casting and T-measure